In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 증강현실 1: GLUT 인터페이스 PyOpenGL

In [ ]:
!pip install pygame
!pip install PyOpenGL
!apt-get install -y freeglut3-dev
!pip install PyOpenGL PyOpenGL_accelerate


import cv2
import numpy as np
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *
import pygame
from pygame.locals import *

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-6).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


## 1. 영상 읽기

* 현재 영상 선택
```python
image_select = 1 #img1
```

In [ ]:
#1
img1 = cv2.imread('/content/drive/MyDrive/Image/homography_source_desired_images(1).jpg')
img2 = cv2.imread('/content/drive/MyDrive/Image/homography_source_desired_images(2).jpg')
imageSize = (img1.shape[1], img1.shape[0])   #(width, height)
image_select = 1 #img1
patternSize = (9, 6)

## 2. Z = 0 평면 세계좌표 생성

In [ ]:
#2: set world(object) coordinates to Z = 0
xN, yN = patternSize         #(9, 6)
mW = np.zeros((xN*yN, 3), dtype=np.float32)   #(54, 3)
# mW points on Z = 0
mW[:, :2] = np.mgrid[0:xN, 0:yN].T.reshape(-1, 2)
mW[:, :2] += 1

## 3. 'K_dist.npz' 파일에서 파라미터 가져오기

  $$
  \mathbf{K} =
  \begin{bmatrix}
  f_x & 0 & c_x \\
  0 & f_y & c_y \\
  0 & 0 & 1
  \end{bmatrix}
  $$
  - $(f_x, f_y$): x, y 방향의 **화소 단위**초점 거리.
  - $(c_x, c_y$): 이미지 센서에서의 주점(principal point)의 좌표.

In [ ]:
#3: load calibration parameters
with np.load('/content/drive/MyDrive/Image/K_dist.npz') as X:
    K, dists, rvecs, tvecs = [X[i] for i in ('K', 'dists', 'rvecs', 'tvecs')]
##dists = np.zeros(5)
print('K=\n', K)
print('dists=', dists)

K=
 [[531.00836922   0.         340.02704252]
 [  0.         531.41264778 233.01782291]
 [  0.           0.           1.        ]]
dists= [[-0.25311424 -0.2797698   0.00264482 -0.0013122   1.19010521]]


## 4. OpenGL 투영 행렬 계산하기

- OpenCV에서는 3D 점을 2D 이미지 평면으로 바로 투영하는 데 중점을 두며, 그 과정에서 **3x4** 행렬을 사용합니다.
- OpenGL에서는 3D 장면을 렌더링하기 위해 **3D 좌표계**를 **NDC(Normalized Device Coordinates)**로 변환한 후, 이를 2D 화면에 매핑하는 과정을 처리하기 때문에 **4x4** 행렬을 사용합니다.

이 때문에 두 시스템은 투영 행렬의 구조와 목적이 다르며, OpenGL에서는 추가적으로 깊이(depth) 정보와 원근 투영을 다루기 위한 4x4 행렬을 사용합니다.

In [ ]:
#4: OpenGL camera parameter, opengl_proj
#4-1:
cx = K[0, 2]
cy = K[1, 2]
fx = K[0, 0]
fy = K[1, 1]

w, h = imageSize
near = 0.1
far = 100.0
#4-2:
opengl_proj = np.array(
    [
        [2 * fx / w, 0.0, (w - 2 * cx) / w, 0.0],
        [0.0, 2 * fy / h, (-h + 2 * cy) / h, 0.0],
        [0.0, 0.0, -(far + near) / (far - near), -2 * far * near / (far - near)],
        [0.0, 0.0, -1.0, 0.0]
    ], dtype=np.float32
)

#opengl_proj[1] *= -1
print('opengl_proj=', opengl_proj)

opengl_proj= [[ 1.6568124   0.         -0.06092681  0.        ]
 [ 0.          2.2142193  -0.0290924   0.        ]
 [ 0.          0.         -1.002002   -0.2002002 ]
 [ 0.          0.         -1.          0.        ]]


## 5. 텍스처 매핑을 위한 left, right, top, bottom 세계 좌표계 단위로 변환.

* mx, my는 단위 길이 당 화소 개수. ( world coords(uint) = image coords(pixel) / pixel per uint)
```python
mx = fx / F    #x-pixel per uint
my = fy / F    #y-pixel per uint
```

In [ ]:
#5: calculate (left, right, top, bottom)
F = 10.0   #set arbitrary focal length, use in #8
mx = fx / F    #x-pixel per uint
my = fy / F    #y-pixel per uint

#convert image coords to unit of woord coords, use in #7
left = cx / mx
right = (w - cx) / mx
top = cy / my
bottom = (h - cy) / my

print('left=', left)
print('right=', right)
print('top=', top)
print('bottom=', bottom)

left= 6.403421532166801
right= 5.6679512966105525
top= 4.384875367213738
bottom= 4.647653346596939


## 6.

* initGL() 함수에서 glLoadTransposeMatrix()로 GL_PROJECTION 투영 행렬에 opengl_proj 행 우선(row-major) 행렬을 로드한다. texture-id를 생성하여 반환한다.

In [ ]:
def initGL():
    glClearColor(0.0, 0.0, 0.0, 0.0)
    glMatrixMode(GL_PROJECTION)
    glLoadTransposeMatrixd(opengl_proj)   # row-major matrix
    ## opengl_proj = np.transpose(opengl_proj)
    ## glLoadMatrixd(opengl_proj) #column-major matrix

    ## proj = glGetFloatv(GL_PROJECTION_MATRIX)
    ## proj = np.transpose(proj)
    ## print('proj=', proj)

    texture_id = glGenTextures(1)
    return texture_id

In [ ]:
#7: texture mapping
def drawBackgroundTexture():
    glBegin(GL_QUADS)
    glTexCoord2f(0.0, 1.0); glVertex3f(-left, bottom, 0.0)
    glTexCoord2f(1.0, 1.0); glVertex3f(right, bottom, 0.0)
    glTexCoord2f(1.0, 0.0); glVertex3f(right, top, 0.0)
    glTexCoord2f(0.0, 0.0); glVertex3f(-left, top, 0.0)
    glEnd()

In [ ]:
#8
def displayImage(image):
    image_size = image.imageSize
    #create texture
    image = cv2.undistort(image, K, dists)
    glBindTexture(GL_TEXTURE_2D, background_texture)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)

    # Using numpy
    iy, ix = image.shape[:2]
    image = image[..., ::-1]  #채널 뒤집기 BGR -> RGB
    image = np.frombuffer(image.tobytes(), dtype = 'uint8', count = image_size)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, ix, iy, 0, GL_RGB, GL_UNSIGNED_BYTE, image)

    # draw background image
    glEnable(GL_TEXTURE_2D)
    glBindTexture(GL_TEXTURE_2D, background_texture)
    glPushMatrix()
    glTranslatef(0.0, 0.0, -F)

    drawBackgroundTexture()
    glPopMatrix()
    glDisable(GL_TEXTURE_2D)


In [ ]:
 #9
def drawCube(bottomFill = True):
    index = [0, 5, 17, 12]
    pts0 = mW[index]   #Z = 0

    if bottomFill:
        glBegin(GL_QUADS)
    else:
        glBegin(GL_LINE_LOOP)

    glColor3f(1.0, 1.0, 0.0)
    for i in range(4):
        glVertex3fv(pts0[i])
    glEnd()

    glColor3f(1.0, 0.0, 0.0)
    pts2 = pts0.copy()
    pts2[:, 2] = -2   #Z = -2
    glBegin(GL_LINE_LOOP)
    for i in range(4):
        glVertex3fv(pts2[i])
    glEnd()

    glColor3f(0.0, 0.0, 1.0)
    glBegin(GL_LINES)
    for i in range(4):
        glVertex3fv(pts0[i])
        glVertex3fv(pts2[i])
    glEnd()

In [ ]:
#10
def displayAxesCube(view_matrix):
    axis3d = np.float32([0, 0, 0], [3, 0, 0] , [0, 3, 0], [0, 0, -3]).reshape(-1, 3)
    glMatrixMode(GL_MODELVIEW)
    glPushMatrix()
    glLoadTransposeMatrixd(view_matrix)

    #draw axes
    glLineWidth(5.0)
    glBegin(GL_LINES)
    glColor3f(0.0, 0.0, 1.0)    #blue
    glVertex3fv(axis3d[0])
    glVertex3fv(axis3d[1])

    glColor3f(0.0, 1.0, 0.0)    #green
    glVertex3fv(axis3d[0])
    glVertex3fv(axis3d[2])

    glColor3f(1.0, 0.0, 0.0)    #red
    glVertex3fv(axis3d[0])
    glVertex3fv(axis3d[3])
    glEnd()
    glPopMatrix()

    #draw cube
    glMatrixMode(GL_MODELVIEW)
    glPushMatrix()
    glLoadTransposeMatrixd(view_matrix)
    drawCube()
    glPopMatrix()

 $$
  [\mathbf{R} \mid \mathbf{t}] =
  \begin{bmatrix}
  r_{11} & r_{12} & r_{13} & t_x \\
  r_{21} & r_{22} & r_{23} & t_y \\
  r_{31} & r_{32} & r_{33} & t_z
  \end{bmatrix}
  $$


In [ ]:
#11
def getViewMatrix(rvec, tvec):
    tvec = tvec.flatten()
    R = cv2.Rodrigues(rvec)[0]
    # Axes Y, Z of OpenGL are the same as -Y, -Z of OpenCV repectively
    view_matrix = np.array([[R[0, 0], R[0, 1], R[0, 2], tvec[0]],
                            [-R[1, 0], -R[1, 1], -R[1, 2], tvec[1]],
                            [-R[2, 0], -R[2, 1], -R[2, 2], tvec[2]],
                            [0.0,        0.0,      0.0,       1.0]])
    return view_matrix

In [ ]:

#12
tx = 0.5
ty = 0.5

def displayFun():
    glClear(GL_COLOR_BUFFER_BIT|GL_DEPTH_BUFFER_BIT)
    #12-1: create texture
    if image_select == 1:
        image = img1
    else:
        image = img2
    displayImage(image)

    #12-2: create view matrix using tvec and rvec
    i = image_select - 1
    tvec = tvecs[i].flatten()
    rvec = rvecs[i].flatten()
    view_matrix = getViewMatrix(rvec, tvec)

    #12-3: display X, Y, Z
    displayAxesCube(view_matrix)

    #12-4: draw glut cube
    glMatrixMode(GL_MODELVIEW)
    glPushMatrix()
    glLoadTransposeMatrixd(view_matrix)
    glTranslatef(tx, ty, -1)   #move on Z = 0
    glScale(1., 1., 2.0)       #Size = (1, 1, 2)

    glColor3f(0.0, 1.0, 0.0)
    glutWireCube(1.0)
    glPopMatrix()

    glColor3f(1.0, 1.0, 1.0)     #white
    glutSwapBuffers()

In [ ]:
#13: handle keyboard events
def keyFun(key, x, y):
    global image_select

    if key == b'\x1b':
        glutDestroyWindow(window_id)
        sys.exit()
    elif key == b'1':
        image_select = 1
    elif key == b'2':
        image_select = 2
    glutPostRedisplay()      #displayFun()

In [ ]:
#14: handle arrow keys
def specialKeyFun(key, x, y):
    global tx, ty

    if key == GLUT_KEY_LEFT:
        ty -= 1.0
    elif key == GLUT_KEY_RIGHT:
        ty += 1.0
    elif key == GLUT_KEY_UP:
        tx -= 1.0
    elif key == GLUT_KEY_DOWN:
        tx += 1.0
    glutPostRedisplay()        #displayFun()

In [ ]:
#15
def main():
    global win_id, background_texture
    glutInit()
    glutInitDisplayMode(GLUT_RGBA|GLUT_DOUBLE|GLUT_DEPTH)
    glutInitWindowSize(640, 480)
    win_id = glutCreateWindow('Augmented Reality')

    background_texture = initGL()
    glutDisplayFunc(displayFun)
    glutKeyboardFunc(keyFun)
    glutSpecialFunc(specialKeyFun)      #arrow key
    glutMainLoop()

if __name__ == '__main__':
    main()

NullFunctionError: Attempt to call an undefined function glutInitDisplayMode, check for bool(glutInitDisplayMode) before calling

In [ ]:
import cv2
import numpy as np
np.set_printoptions(precision = 2, suppress = True)

#1
from OpenGL.GL import *
from OpenGL.GLU import *

import pygame
from pygame.locals import *
from objloader import OBJ

ModuleNotFoundError: No module named 'objloader'